In [ ]:
import numpy
import matplotlib.pyplot as plt
from scipy.signal.spectral import lombscargle

# Test with Artificial Signal

Start with regularly sampled signal

In [ ]:
xvar = numpy.arange(0,128,1.0)

fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
# Test averaging
averages = 10
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5 +numpy.sin(2*numpy.pi * 0.08 * xvar)

    #plt.plot(xvar,sine_test)
    #plt.show()

    hamm_sine = numpy.hamming(128) * sine_test
    ## FFT
    fft += numpy.fft.fft(hamm_sine)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,0.64,0.01)
    lmb += lombscargle(xvar, hamm_sine,freqs)

    
plt.semilogy(xvar,numpy.square(numpy.abs(fft/averages)))    
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))


## Irregularly Sampled Signal

A single flagged out channel

In [ ]:
randn = numpy.random.randint(0,128)
fft = numpy.zeros(shape=(128),dtype=numpy.complex128)
lmb = numpy.zeros(shape=(64))
averages=1000
for i in numpy.arange(averages):

    sine_test = numpy.sin(2*numpy.pi * 0.04 * xvar) + numpy.random.randn(128)/5+numpy.sin(2*numpy.pi * 0.06 * xvar)
    hamm_sine = numpy.hamming(128) * sine_test
    hamm_sine_test_fft = hamm_sine.copy()
    
    hamm_sine_test_fft[randn] = 0.0
    hamm_sine_test_lmb = hamm_sine.copy()
    hamm_sine_test_lmb = numpy.delete(hamm_sine_test_lmb,randn)
    xvar_lmb = xvar.copy()
    xvar_lmb = numpy.delete(xvar_lmb, randn)
    #plt.plot(xvar,sine_test)
    #plt.show()

    
    ## FFT
    fft += numpy.fft.fft(hamm_sine_test_fft)/128
    freqs = numpy.fft.fftfreq(len(fft))
    
    ## Lombscargle
    freqs = numpy.arange(0.0001,0.64,0.01)
    lmb += lombscargle(xvar_lmb, hamm_sine_test_lmb,freqs)/128

    
plt.semilogy(xvar,numpy.square(numpy.abs(fft/averages)))    
plt.semilogy(numpy.arange(0,64,1),numpy.abs(lmb/averages))

In [ ]:
%run procps.py
%run plotprelude.py

In [ ]:
def lombscarglecsd(x1, x2, y1, y2, freqs, precenter=False, normalize=False):
    """
    """
    assert(x1 is x2)
    assert(x1.ndim == 1)
    assert(y1.ndim == 1)
    assert(freqs.ndim==1)

    pgram1 = lombscargle(x1, y1 - y1.mean(), freqs,precenter=precenter,normalize=normalize)
    pgram2 = lombscargle(x2, y2 - y2.mean(), freqs,precenter=precenter,normalize=normalize)

    if normalize:
        pgram1 *= 2 / np.dot(y1,y1)
        pgram2 *= 2 / np.dot(y2,y2)

    return numpy.conjugate(pgram1) * pgram2 # I assume this still holds for the lombscargle periodgram?


In [ ]:
fin = numpy.load("/rds/project/bn204/rds-bn204-asterics/jck42/BiSpectrum/Binned_Data/EQ14/XX/output_bin.npz")
cl = fin['closures']

eq14_1 = mdays(cl[:,0:8], 1)
eq14_2 = mdays(cl[:,8:], 1)
eq14 = numpy.concatenate((eq14_1, eq14_2), axis=1)



In [ ]:
x1=psXmedCTimCTri(numpy.exp(1j*(eq14[100:250,:,:,515:775])),
                       window="hamming")
pp_ps(x1,title="EQ14, XX, Times 100:250, Chans 515:775")

In [ ]:
# Try zeroing some channels

eq14c = eq14.copy()
eq14c[:,:,:,560] = 0.0
x1=psXmedCTimCTri(numpy.exp(1j*(eq14c[100:250,:,:,515:775])),
                       window="hamming")
pp_ps(x1,title="EQ14, XX, Times 100:250, Chans 515:775")

In [ ]:
#Try using lombscargle

closure_subset = eq14c[100:250,:,:,515:775]
closure_subset = numpy.delete(closure_subset, 45,axis=3)
print(closure_subset.shape)

xvar = numpy.arange(0,260).astype(float)
xvar = numpy.delete(xvar,45)

freqs = numpy.linspace(0.00000001,260,128).astype(float)


print(closure_subset[:,0,2,:].shape)
lmb = lombscargle(xvar, closure_subset[75,0,2,:],freqs)
lmbcsd = lombscarglecsd(xvar,xvar,closure_subset[75,0,2,:],closure_subset[75,1,2,:],freqs)

pp_ps(lmbcsd)
pp_ps(lmb)